In [ ]:
# 特征工程 
#
# 配置描述     |基础运算            |联合统计       |隐匿查询          |安全求交        |特征工程           |联合建模   |联合预测 
# 数据方       |两方, 三方          |两方          |两方              |两方            |两方              |两方       |两方 
# 算法类型     |加法, 乘法, 比较    |方差, 中位数   |高效率, 高隐匿性   |平衡, 非平衡    |WOE和IV计算        |LR        |LR
# 测试数据集   |数据集α             |数据集α       |数据集β            |数据集γ        |epsilon           |epsilon    |epsilon

import os
path = './ds'
assert os.path.exists(path)

In [ ]:
%%time

import pyarrow as pa 
import pyarrow.dataset as ds
import pyarrow.compute as pc
import pandas as pd
import numpy as np

ds1 = ds.dataset(f'{path}/ds_epsilon1_test')
epsilon_y = ds1.to_table(columns={'0': ds.field('0')})['0'].to_pandas()
proj = {col:ds.field(col) for col in ds1.schema.names}
proj.pop('0')
epsilon1 = ds1.to_table(columns=proj)
epsilon2 = ds.dataset(f'{path}/ds_epsilon2_test').to_table()
epsilon_X = pd.DataFrame({col: data.to_pandas() for col, data 
        in zip(epsilon1.column_names+epsilon2.column_names, epsilon1.columns+epsilon2.columns)})

In [ ]:
%%time

N = 10
woe_iv = pd.DataFrame(index=pd.RangeIndex(0, N))
for col in epsilon_X.columns:
    bins = pd.cut(epsilon_X[col], N)
    df_woe_iv = pd.crosstab(bins, epsilon_y, normalize='columns') \
                  .rename(mapper={1:'P', -1:'N'}, axis=1) \
                  .assign(woe=lambda dfx: np.log(dfx['P'] / dfx['N'])) \
                  .assign(iv=lambda dfx: np.sum(dfx['woe']*(dfx['P']-dfx['N']))) \
                  .reset_index() \
                  .rename(mapper=lambda cn: f"{cn}_cut{col}", axis=1)
    woe_iv = woe_iv.join(df_woe_iv)
    bins_q = pd.qcut(epsilon_X[col], N)
    df_woe_iv_q = pd.crosstab(bins_q, epsilon_y, normalize='columns') \
                    .rename(mapper={1:'P', -1:'N'}, axis=1) \
                    .assign(woe=lambda dfx: np.log(dfx['P'] / dfx['N'])) \
                    .assign(iv=lambda dfx: np.sum(dfx['woe']*(dfx['P']-dfx['N']))) \
                    .reset_index() \
                    .rename(mapper=lambda cn: f"{cn}_qcut{col}", axis=1)
    woe_iv = woe_iv.join(df_woe_iv_q)

In [ ]:
from collections import Counter
assert not [(k, v) for k,v in Counter(woe_iv.columns).items() if v>1]

# currently raise ArrowNotImplementedError: 
# Unsupported cast to extension<pandas.interval<ArrowIntervalType>> from dictionary<values=extension<pandas.interval<ArrowIntervalType>>, indices=int8, ordered=1>
# ds.write_dataset(pa.Table.from_pandas(woe_iv), f'{path}/epsilon_iv', format='parquet')

In [ ]:
woe_iv["1_q